![](https://i.pinimg.com/564x/02/9d/32/029d32a51d8e5d485a9e30cc00ff4026.jpg)

# Jacobians

Kevin Walchko, Phd

---

As an application, in the field of control engineering the use of Jacobian matrices allows the local (approximate) linearisation of non-linear systems around a given equilibrium point and so allows the use of linear systems techniques, such as the calculation of eigenvalues (and thus allows an indication of the type of the equilibrium point).

Jacobians are also used in the estimation of the internal states of non-linear systems in the construction of the extended Kalman filter, and also if the extended Kalman filter is to be used to provide joint state and parameter estimates for a linear system (since this is a non-linear system analysis due to the products of what are then effectively inputs and outputs of the system).

The Jacobian matrix of $f$ is defined to be an $m \times n$ matrix, denoted by $J$, whose $(i,j)$th entry is $\mathbf J_{ij} = \frac{\partial f_i}{\partial x_j}$, or explicitly 

$$
\mathbf J = \begin{bmatrix}
    \dfrac{\partial \mathbf{f}}{\partial x_1} & \cdots & \dfrac{\partial \mathbf{f}}{\partial x_n} \end{bmatrix}
= \begin{bmatrix}
    \dfrac{\partial f_1}{\partial x_1} & \cdots & \dfrac{\partial f_1}{\partial x_n}\\
    \vdots & \ddots & \vdots\\
    \dfrac{\partial f_m}{\partial x_1} & \cdots & \dfrac{\partial f_m}{\partial x_n} \end{bmatrix}
$$

## References

- wikipedia: [Jacobian Matrix](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant#Example_1)
- stackexchange [Calculate Jacobian](https://stackoverflow.com/a/54954446/5374768)
- math.stackexchance: [Jacobian uses](https://math.stackexchange.com/a/36648)

## Example

[Ref](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant#Example_1)

Given the following non-linear function, find the Jacobian and its determinant.

$$
\mathbf f\left(\begin{bmatrix} x\\y\end{bmatrix}\right) = \begin{bmatrix} f_1(x,y)\\f_2(x,y)\end{bmatrix} =
  \begin{bmatrix}  x^2 y \\5 x + \sin y 
  \end{bmatrix}
$$

Then we have

$$
f_1(x, y) = x^2 y
$$

and

$$f_2(x, y) = 5 x + \sin y$$

and the Jacobian matrix of $f$ is

$$
\mathbf J_{\mathbf f}(x, y) = \begin{bmatrix}
  \dfrac{\partial f_1}{\partial x} & \dfrac{\partial f_1}{\partial y}\\[1em]
  \dfrac{\partial f_2}{\partial x} & \dfrac{\partial f_2}{\partial y} \end{bmatrix}
= \begin{bmatrix}
  2 x y & x^2    \\
  5     & \cos y \end{bmatrix}
$$

and the Jacobian determinant is
$\det(\mathbf J_{\mathbf f}(x, y)) = 2 x y \cos y - 5 x^2$

In [218]:
import numpy as np
from math import sin, cos

In [258]:
def perf(x, a):
    print("Result:")
    print(x, "\n")
    print("Error:", np.linalg.norm(a-x))

In [155]:
def func(x):
    """
    Some cool non-linear function
    f = [x**2y; 5x+sin(y)]
    """
    f1 = x[0]**2*x[1]
    f2 = 5*x[0]+sin(x[1])
    
    return np.array([f1,f2])

In [247]:
# state
x = np.array([5,2])

In [256]:
def dfunc(x):
    """
    The manually calculated Jacobian of the cool function
    f = [x**2y; 5x+sin(y)]
    """
    ret = [
        [2*x[0]*x[1], x[0]**2],
        [5, cos(x[1])]
    ]
    
    return np.array(ret)

# here is the answer we expect
ans = dfunc(x)
print(ans)

[[20.         25.        ]
 [ 5.         -0.41614684]]


## Jacobian Class

These classes perform a *center* and *forward* numerical differentiation of a function to produce the Jacobian. The original method was found on-line and then I optimized to make it more efficient

In [242]:
import attr

@attr.s(slots=True)
class Jacobian:
    f=attr.ib() # function
    
    # cache some variables to save processing time
    n=attr.ib(init=False, default=None)
    jac=attr.ib(init=False, default=None)
    
    
@attr.s(slots=True)
class JacobianForward(Jacobian):
    def __call__(self, x, dx=1e-8):
        if self.n is None:
            self.n = len(x)
            self.jac = np.zeros((self.n, self.n))
        func = self.f(x)
        
        for j in range(self.n):
            Dxj = (abs(x[j])*dx if x[j] != 0 else dx)
            d = np.zeros(self.n)
            d[j] = Dxj
            self.jac[:, j] = (self.f(x+d) - func)/Dxj
        return self.jac
    
    
@attr.s(slots=True)
class JacobianCenter(Jacobian):
    def __call__(self, x, dx=1e-8):
        if self.n is None:
            self.n = len(x)
            self.jac = np.zeros((self.n, self.n))
        for j in range(self.n):
            Dxj = (abs(x[j])*dx if x[j] != 0 else dx)
            d = np.zeros(self.n)
            d[j] = Dxj
            self.jac[:, j] = (self.f(x+d) - self.f(x-d))/(2*Dxj)
        return self.jac
    
    
@attr.s(slots=True)
class JacobianCenterOriginal(Jacobian):
    def __call__(self, x, dx=1e-8):
        n = len(x)
        jac = np.zeros((n, n))
        for j in range(n):  # through columns to allow for vector addition
            Dxj = (abs(x[j])*dx if x[j] != 0 else dx)
            x_plus = [(xi if k != j else xi + Dxj) for k, xi in enumerate(x)]
            x_minus = [(xi if k != j else xi - Dxj) for k, xi in enumerate(x)]
            jac[:, j] = (self.f(x_plus) - self.f(x_minus))/(2*Dxj)
        return jac

In [260]:
# original
jj = JacobianCenterOriginal(func)
perf(jj(x), ans)
%timeit jj(x)

Result:
[[19.99999988 25.00000011]
 [ 4.99999999 -0.41614685]] 

Error: 1.680392819408006e-07
62.3 µs ± 3.11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [261]:
# center - same as original, but faster
jj = JacobianCenter(func)
perf(jj(x), ans)
%timeit jj(x)

Result:
[[19.99999988 25.00000011]
 [ 4.99999999 -0.41614685]] 

Error: 1.680392819408006e-07
37 µs ± 2.86 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [262]:
# forward 
jj = JacobianForward(func)
perf(jj(x), ans)
%timeit jj(x)

Result:
[[19.99999995 24.99999994]
 [ 4.99999999 -0.41614676]] 

Error: 1.1102231361837889e-07
34 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Conclusion

Based on the above, my optimizations produced less than a 2x increase (which is really good). I believe if the state vector was much larger, then the performance increase would improve.

Also, there is little difference in the error, at least for this example, between the central and forward Jacobian. The forward Jacobian is just slightly faster, so depending on the situation, you can trad-off speed for accuracy. Again, the performance difference will increase as state space increases because forward avoids 1 call per state variable in the Jacobian loop.